In [ ]:
# Reload the notebook automatically
%load_ext autoreload
%autoreload 2

# Import

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
from IPython.display import clear_output

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src import ROOT, parse
from src.models import get_model
from src.datasets.transforms import *
from src.datasets import get_dataset, get_dataloader
from src.utils import *

cfg_dir = 'mlcv-exp/data/cfg/'

# Models

In [ ]:
dataset = 'fpha'
model_name = 'yolov2_bbox'
exp = 'exp2'
epoch = 200
cfg_name = '{}_{}_{}.cfg'.format(dataset, model_name, exp)
cfg = parse(Path(ROOT)/cfg_dir/model_name/cfg_name)
cfg['aug'] = None
cfg['batch_size'] = 1
cfg['shuffle'] = None
model_1 = get_model(cfg, 'test', epoch)

In [ ]:
dataset = 'ek'
model_name = 'hpo_ar'
exp = 'exp1'
epoch = 2
cfg_name = '{}_{}_{}.cfg'.format(dataset, model_name, exp)
cfg = parse(Path(ROOT)/cfg_dir/model_name/cfg_name)
cfg['aug'] = None
cfg['batch_size'] = 1
cfg['shuffle'] = None
model_2 = get_model(cfg, 'test', epoch)

In [ ]:
model_1.net.train()
model_2.net.eval()
clear_output()

# Mix features

In [ ]:
img_path = Path('EPIC_KITCHENS_2018')/'EK_frames'/'P20/P20_03_21943_open_drawer'/'img_{:05d}.jpg'.format(100)
img = get_img_dataloader(str(Path(ROOT)/img_path), 416)
img = img.unsqueeze(0).cuda()

In [ ]:
do_sum = 0
def mix_feat(x_1, x_2):
    return (x_1 + x_2)/2.0

In [ ]:
spatial_attn_map            = []
spatial_attn_map_block0     = []
spatial_attn_map_block1_1   = []
spatial_attn_map_block1_2   = []

x_1 = img.clone()
x_2 = img.clone()

for layer1, layer2 in zip(model_1.net.backbone.block0, model_2.net.backbone.block0):
    x_1 = layer1(x_1)
    x_2 = layer2(x_2)
    if do_sum:
        x_1 = mix_feat(x_1, x_2)
    spatial_attn_map_block0.append(x_1) 
out0 = x_1.clone()
spatial_attn_map.append(spatial_attn_map_block0)

x_1 = out0.clone()
for layer1, layer2 in zip(model_1.net.backbone.block1_1, model_2.net.backbone.block1_1):
    x_1 = layer1(x_1)
    x_2 = layer2(x_2)
    if do_sum:
        x_1 = mix_feat(x_1, x_2)
    spatial_attn_map_block1_1.append(x_1) 
out1_1 = x_1.clone()
spatial_attn_map.append(spatial_attn_map_block1_1)

x_1 = out0.clone()
x_2_2 = out0.clone()
for layer1, layer2 in zip(model_1.net.backbone.block1_2, model_2.net.backbone.block1_2):
    x_1 = layer1(x_1)
    x_2_2 = layer2(x_2_2)
    if do_sum:
        x_1 = mix_feat(x_1, x_2_2)
    spatial_attn_map_block1_2.append(x_1) 
out1_2 = x_1.clone()
spatial_attn_map.append(spatial_attn_map_block1_2)

out1 = torch.cat([out1_1, out1_2], 1)
x_2 = torch.cat([x_2, x_2_2], 1)
spatial_attn_map.append([out1])

x_1 = out1.clone()
for layer1, layer2 in zip(model_1.net.backbone.block2, model_2.net.backbone.block2):
    x_1 = layer1(x_1)
    x_2 = layer2(x_2)
    if do_sum:
        x_1 = sum_feat(x_1, x_2)
out2 = x_1.clone()
spatial_attn_map.append([out2])

pred = model_1.net.conv_out(out2)
spatial_attn_map.append([pred])

pred = pred.cpu()

In [ ]:
img_copy    = img.clone()
img_copy = ImgToNumpy()(img_copy.cpu())[0]
fig, ax = plt.subplots()
ax.imshow(img_copy)
plt.show()

# Spatial Attention Map

In [ ]:
func = abs_sum_p(2)

In [ ]:
proc_attn_map = [[x[0].detach().cpu().numpy() for x in block] for block in spatial_attn_map]
proc_attn_map = [[func(x) for x in block] for block in proc_attn_map]

In [ ]:
idx = 4
block = proc_attn_map[idx]
for i in range(len(block)):
    print(i)
    fig, ax = plt.subplots()
    block_rsz = cv2.resize(block[i][:, :, np.newaxis], img_copy.shape[:2])
    ax.imshow(block_rsz, cmap='jet')
    ax.imshow(img_copy, alpha=0.5)
    plt.axis('off')
    plt.show()

# Detect

In [ ]:
max_boxes   = 2

box         = get_region_boxes(pred,
                            model_1.pred_conf_thresh,
                            0,
                            model_1.anchors,
                            model_1.num_anchors,
                            is_cuda=False)[0]

boxes       = nms_torch(box, model_1.pred_nms_thresh)

all_boxes   = np.zeros((max_boxes, 5))
if len(boxes) != 0:
    if len(boxes) > len(all_boxes):
        fill_range = len(all_boxes)
    else:
        fill_range = len(boxes)

    for i in range(fill_range):
        box             = boxes[i]
        all_boxes[i]    = (float(box[0]), float(box[1]),
                            float(box[2]), float(box[3]), 
                            float(box[4]))

bbox_pred_1 = all_boxes[0]
bbox_pred_2 = all_boxes[1]

bbox_pred_1_show     = bbox_pred_1.copy()
bbox_pred_1_show[0]  = bbox_pred_1_show[0]*img_copy.shape[1]
bbox_pred_1_show[1]  = bbox_pred_1_show[1]*img_copy.shape[0]
bbox_pred_1_show[2]  = bbox_pred_1_show[2]*img_copy.shape[1]
bbox_pred_1_show[3]  = bbox_pred_1_show[3]*img_copy.shape[0]

bbox_pred_2_show     = bbox_pred_2.copy()
bbox_pred_2_show[0]  = bbox_pred_2_show[0]*img_copy.shape[1]
bbox_pred_2_show[1]  = bbox_pred_2_show[1]*img_copy.shape[0]
bbox_pred_2_show[2]  = bbox_pred_2_show[2]*img_copy.shape[1]
bbox_pred_2_show[3]  = bbox_pred_2_show[3]*img_copy.shape[0]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img_copy)
print(bbox_pred_1_show[4])
draw_bbox(ax, bbox_pred_1_show, 'r')
# draw_bbox(ax, bbox_pred_2_show, 'r')
plt.show()